In [76]:
import pandas as pd
from sqlalchemy import create_engine
import scipy.stats as sts
import numpy as np
import datetime
import psycopg2

In [77]:
# clean df with relevant columns - country,iso_code, date, daily_vaccinations, vaccines,source_name

In [78]:
csv_file = '../Resources/country_vaccinations.csv'
world_vac_df = pd.read_csv(csv_file)

In [118]:
county_df = world_vac_df[['country', 'iso_code', 'date', 'daily_vaccinations',
                       'vaccines', 'source_name']]
world_vac_df.sample(50)



,country,iso_code,date,daily_vaccinations,vaccines,source_name
1511,Scotland,NaN,2021-01-25,22213.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom
1076,Malta,MLT,2021-01-10,396.0,Pfizer/BioNTech,Ministry of Health
1918,Wales,NaN,2021-01-21,12318.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom
555,Estonia,EST,2021-01-24,1112.0,Pfizer/BioNTech,National Health Board
47,Austria,AUT,2021-01-20,9833.0,Pfizer/BioNTech,Ministry of Health
1628,Slovenia,SVN,2021-01-20,2415.0,Pfizer/BioNTech,"National Institute of Public Health, via Sledi..."
1588,Slovakia,SVK,2021-01-09,4456.0,Pfizer/BioNTech,Ministry of Health
1400,Romania,ROU,2021-01-22,28895.0,Pfizer/BioNTech,Government of Romania
805,India,IND,2021-01-28,269217.0,"Covaxin, Oxford/AstraZeneca",Ministry of Health
994,Lithuania,LTU,2020-12-28,2785.0,"Moderna, Pfizer/BioNTech",Ministry of Health


In [117]:
csv_file = '../Resources/covid_19_clean_complete.csv'
covid_case_df = pd.read_csv(csv_file)
covid_case_df = covid_case_df[['Country/Region','Date','Confirmed','WHO Region']]
covid_case_df = covid_case_df.rename(columns={'Country/Region': 'Country'})
covid_case_df.sample(30)
##Cleanup
    #Reduce to only relevant columns
    #change the column:Country/Region to Country
#changes each day for countries

    



,Country,Date,Confirmed,WHO Region
17310,Costa Rica,2020-03-28,295,Americas
42771,Zimbabwe,2020-07-03,625,Africa
1693,Hungary,2020-01-28,0,Europe
16230,China,2020-03-24,990,Western Pacific
8322,Mozambique,2020-02-22,0,Africa
26994,France,2020-05-04,6,Europe
24697,Nepal,2020-04-25,49,South-East Asia
13498,Saint Lucia,2020-03-13,0,Americas
33465,China,2020-05-29,328,Western Pacific
44114,Antigua and Barbuda,2020-07-09,73,Americas


In [112]:

  covid_case_df.sample(30)

,Country,Date,Confirmed,WHO Region
24651,Georgia,2020-04-25,456,Europe
45740,China,2020-07-15,932,Western Pacific
28826,Gambia,2020-05-11,22,Africa
36383,Finland,2020-06-09,7025,Europe
25174,Germany,2020-04-27,158758,Europe
35079,France,2020-06-04,556,Europe
38251,Lebanon,2020-06-16,1473,Eastern Mediterranean
1002,United Kingdom,2020-01-25,0,Europe
9711,China,2020-02-28,252,Western Pacific
10788,Cuba,2020-03-03,0,Americas


In [113]:
csv_file = '../Resources/country_vaccinations.csv'
country_df = pd.read_csv(csv_file)
country_df = country_df[['date','country','iso_code','daily_vaccinations_per_million','vaccines','source_name']]


In [115]:
 country_df.sample(30)

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
399,Cyprus,CYP,2021-01-25,NaN,NaN,NaN,NaN,1061.0,NaN,NaN,NaN,1211.0,Pfizer/BioNTech,Ministry of Health,https://cyprus-mail.com/2021/01/30/coronavirus...
682,Gibraltar,GIB,2021-01-23,9807.0,9807.0,NaN,930.0,566.0,29.11,29.11,NaN,16800.0,Pfizer/BioNTech,Government of Gibraltar,https://twitter.com/GibraltarGov/status/135625...
957,Latvia,LVA,2020-12-28,595.0,NaN,NaN,NaN,NaN,0.03,NaN,NaN,NaN,Pfizer/BioNTech,National Health Service,https://data.gov.lv/dati/eng/dataset/covid19-v...
1850,United States,USA,2021-01-04,4563260.0,4563260.0,NaN,NaN,348017.0,1.36,1.36,NaN,1041.0,"Moderna, Pfizer/BioNTech",Centers for Disease Control and Prevention,https://covid.cdc.gov/covid-data-tracker/#vacc...
174,Bulgaria,BGR,2021-01-05,5448.0,5448.0,NaN,NaN,533.0,0.08,0.08,NaN,77.0,"Moderna, Pfizer/BioNTech",Ministry of Health,https://coronavirus.bg/bg/statistika
1041,Luxembourg,LUX,2021-01-10,NaN,NaN,NaN,NaN,62.0,NaN,NaN,NaN,99.0,Pfizer/BioNTech,Government of Luxembourg,https://data.public.lu/fr/datasets/r/a1038784-...
29,Argentina,ARG,2021-01-25,292386.0,260122.0,32264.0,363.0,9046.0,0.65,0.58,0.07,200.0,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
1499,Scotland,NaN,2021-01-13,211397.0,208207.0,3190.0,16442.0,10597.0,3.87,3.81,0.06,1940.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
1187,Northern Ireland,NaN,2020-12-20,16068.0,16068.0,NaN,NaN,1778.0,0.85,0.85,NaN,939.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
605,France,FRA,2021-01-04,1800.0,1800.0,NaN,1359.0,250.0,0.00,0.00,NaN,4.0,Pfizer/BioNTech,Public Health France,https://www.data.gouv.fr/fr/datasets/donnees-r...


In [132]:
csv_file = '../Resources/country_vaccinations.csv'
country_df = pd.read_csv(csv_file)
country_df = country_df[['date','country','iso_code','daily_vaccinations_per_million','vaccines','source_name']]


In [88]:
country_df.sample(30)

,date,country,iso_code,daily_vaccinations_per_million,vaccines,source_name
1142,2021-01-26,Myanmar,MMR,NaN,Oxford/AstraZeneca,Ministry of Health
906,2021-01-20,Israel,ISR,16554.0,"Moderna, Pfizer/BioNTech",Government of Israel
100,2021-02-01,Bahrain,BHR,890.0,"Pfizer/BioNTech, Sinopharm",Ministry of Health
440,2021-02-01,Czechia,CZE,882.0,"Moderna, Pfizer/BioNTech",Ministry of Health
187,2021-01-18,Bulgaria,BGR,113.0,"Moderna, Pfizer/BioNTech",Ministry of Health
1331,2021-01-25,Poland,POL,1064.0,Pfizer/BioNTech,Ministry of Health
1729,2021-01-15,Switzerland,CHE,442.0,"Moderna, Pfizer/BioNTech",Federal Office of Public Health
1214,2021-01-16,Northern Ireland,NaN,4206.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom
1899,2021-01-02,Wales,NaN,624.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom
687,2021-01-28,Gibraltar,GIB,18581.0,Pfizer/BioNTech,Government of Gibraltar


In [120]:
connection_string = "postgres:Logan5811@localhost:5432/ETL"
engine = create_engine(f'postgresql://{connection_string}')

In [121]:
 # Confirm tables
engine.table_names()

['airport', 'country']

In [144]:
country_df.to_sql(name='country', con=engine, if_exists='append', index=False)

In [154]:
new_covid_case_df.to_sql(name='airport', con=engine, if_exists='append', index=False)